# 等值线

本节介绍如何使用 cedarkit-maps 绘制 500hPa 位势高度等值线图。

首先导入需要使用的包。包括：

- 数据结构：numpy, pandas
- cedarkit 工具套件
    - 数据准备：reki
        - 数据查找函数： `find_local_file`
        - GRIB2文件要素加载函数：`load_field_from_file`
    - 气象可视化：cedarkit.maps
        - 绘图板：`Panel`
        - 底图布局：`EastAsiaMapDomain`
        - 绘图样式：`ContourStyle`, `ContourLabelStyle`

In [ ]:
import numpy as np
import pandas as pd

from reki.data_finder import find_local_file
from reki.format.grib.eccodes import load_field_from_file

from cedarkit.maps.style import ContourStyle, ContourLabelStyle
from cedarkit.maps.chart import Panel
from cedarkit.maps.domains import EastAsiaMapDomain

设置绘图的数据参数，使用 CMA-GFS 2024 年 4 月 1 日 00 时次 024 时效数据。

In [ ]:
system_name = "CMA-GFS"
data_type = "cma_gfs_gmf/grib2/orig"
start_time = pd.to_datetime("2024-04-01 00:00:00")
forecast_time = pd.to_timedelta("24h")

## 加载数据

获取 GRIB2 数据路径

In [ ]:
file_path = find_local_file(
    data_type,
    start_time=start_time,
    forecast_time=forecast_time,
)
file_path

加载 500 hPa 位势高度场。位势高度在 ecCodes 的内置要素名为 `gh`

In [ ]:
h_500_field = load_field_from_file(
    file_path,
    parameter="gh",
    level_type="pl",
    level=500
) / 10.0
h_500_field

## 配置

定义等值线层次

In [ ]:
h_contour_lev = np.linspace(500, 588, endpoint=True, num=23)
h_contour_lev

定义等值线颜色

In [ ]:
h_contour_colos = "blue"

定义线宽

In [ ]:
h_linewidths = np.where(h_contour_lev == 588, 1.4, 0.7)

定义等值线样式 (`ContourStyle`)，包括等值线标签样式 (`ContourLabelStyle`)

In [ ]:
hgt_style = ContourStyle(
    levels=h_contour_lev,
    colors="blue",
    linewidths=h_linewidths,
    label=True,
    label_style=ContourLabelStyle(
        colors="black",
        fontsize=8,
    )
)

## 绘制

创建中国区域底图布局

In [ ]:
domain = EastAsiaMapDomain()

绘制等值线，设置标题

In [ ]:
panel = Panel(domain=domain)
panel.plot(h_500_field, style=hgt_style)

domain.set_title(
    panel=panel,
    graph_name="500 hPa Height(10gpm)",
    system_name=system_name,
    start_time=start_time,
    forecast_time=forecast_time,
)
panel.show()